In [1]:
from nltk.corpus import gutenberg, stopwords
import nltk
from nltk.corpus import stopwords
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, SpatialDropout1D
from keras.layers.embeddings import Embedding
from keras.utils import np_utils

# Others
import string
import numpy as np
import pandas as pd

from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

import re
import os

# set seed for reproducibility
np.random.seed(0)

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# gutenberg.fileids()

# for fileid in gutenberg.fileids():
#     num_chars = len(gutenberg.raw(fileid)) # Tính số lượng ký tự
#     num_words = len(gutenberg.words(fileid)) # Tính số lượng từ đơn
#     num_sents = len(gutenberg.sents(fileid)) # Tính số lượng câu.
#     num_vocab = len(set([w.lower() for w in gutenberg.words(fileid)])) # Tính số lượng từ vựng trong văn bản
#     print(int(num_chars / num_words), int(num_words / num_sents), int(num_words / num_vocab), fileid)

In [3]:
all_files = os.listdir("book/")

In [4]:
arr = []
one_book = []
content = ''
path = 'book/'
for b in all_files[0:400]:
    one_book = []
    path = 'book/'
    one_book.append(b.split('___')[0])
    one_book.append(b)
    
    path += b
    with open(path, 'r') as myfile:
        content = myfile.read().replace('\n', ' ')
    one_book.append(content)
    arr.append(one_book)

In [5]:
columns = ['author', 'file_name', 'text']
df = pd.DataFrame(arr, columns=columns)

In [6]:
df.shape

(400, 3)

In [7]:
df.head()

,author,file_name,text
0,Abraham Lincoln,Abraham Lincoln___Lincoln Letters.txt,LINCOLN LETTERS By Abraham Lincoln Publish...
1,Abraham Lincoln,Abraham Lincoln___Lincoln's First Inaugural Ad...,"Lincoln's First Inaugural Address March 4, 18..."
2,Abraham Lincoln,Abraham Lincoln___Lincoln's Gettysburg Address...,"Lincoln's Gettysburg Address, given November..."
3,Abraham Lincoln,"Abraham Lincoln___Lincoln's Inaugurals, Addres...",Longman's English Classics LINCOLN'S INAUGUR...
4,Abraham Lincoln,Abraham Lincoln___Lincoln's Second Inaugural A...,"Lincoln's Second Inaugural Address March 4, 1..."


In [8]:
def get_data_of_file(row):
    file_name = row.file_name
    text = row.text
    
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    row['text'] = text
    return row
df = df.apply(get_data_of_file, axis=1)

In [9]:
df.head()

,author,file_name,text
0,Abraham Lincoln,Abraham Lincoln___Lincoln Letters.txt,LINCOLN LETTERS By Abraham Lincoln Published ...
1,Abraham Lincoln,Abraham Lincoln___Lincoln's First Inaugural Ad...,Lincoln First Inaugural Address March 4 1861 ...
2,Abraham Lincoln,Abraham Lincoln___Lincoln's Gettysburg Address...,Lincoln Gettysburg Address given November 19 ...
3,Abraham Lincoln,"Abraham Lincoln___Lincoln's Inaugurals, Addres...",Longman English Classics LINCOLN S INAUGURALS...
4,Abraham Lincoln,Abraham Lincoln___Lincoln's Second Inaugural A...,Lincoln Second Inaugural Address March 4 1865...


In [10]:
df.shape

(400, 3)

In [11]:
le = LabelEncoder()
df['author_LabelEncoded'] = le.fit_transform(df.author)

In [12]:
df.head(15)

,author,file_name,text,author_LabelEncoded
0,Abraham Lincoln,Abraham Lincoln___Lincoln Letters.txt,LINCOLN LETTERS By Abraham Lincoln Published ...,0
1,Abraham Lincoln,Abraham Lincoln___Lincoln's First Inaugural Ad...,Lincoln First Inaugural Address March 4 1861 ...,0
2,Abraham Lincoln,Abraham Lincoln___Lincoln's Gettysburg Address...,Lincoln Gettysburg Address given November 19 ...,0
3,Abraham Lincoln,"Abraham Lincoln___Lincoln's Inaugurals, Addres...",Longman English Classics LINCOLN S INAUGURALS...,0
4,Abraham Lincoln,Abraham Lincoln___Lincoln's Second Inaugural A...,Lincoln Second Inaugural Address March 4 1865...,0
5,Abraham Lincoln,Abraham Lincoln___Speeches and Letters of Abra...,SPEECHES LETTERS OF ABRAHAM LINCOLN 1832 - 18...,0
6,Abraham Lincoln,Abraham Lincoln___State of the Union Addresses...,State of the Union Address Abraham Lincoln De...,0
7,Abraham Lincoln,Abraham Lincoln___The Emancipation Proclamatio...,THE EMANCIPATION PROCLAMATION : By the Presid...,0
8,Abraham Lincoln,Abraham Lincoln___The Life and Public Service ...,NOTE After lying buried for almost three quar...,0
9,Abraham Lincoln,Abraham Lincoln___The Writings of Abraham Linc...,THE PAPERS AND WRITINGS OF ABRAHAM LINCOLN VO...,0


### TF-IDF

In [13]:
vectorizer = TfidfVectorizer(sublinear_tf=True, use_idf =True, lowercase=True, strip_accents='ascii', stop_words = 'english')

In [14]:
XText_tfidf = vectorizer.fit_transform(df['text'])

In [15]:
XText_tfidf.shape

(400, 133346)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(XText_tfidf, df.author_LabelEncoded, random_state = 0)

In [17]:
clf = MultinomialNB().fit(X_train, y_train)

In [18]:
y_score = clf.predict(X_test)

In [19]:
n_right = 0
for i in range(len(y_score)):
    if y_score.tolist()[i] == y_test.tolist()[i]:
        n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(y_test)) * 100)))

Accuracy: 41.00%


In [20]:
print(classification_report(y_test, y_score))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00        10
          3       0.00      0.00      0.00         1
          4       0.00      0.00      0.00         1
          6       0.00      0.00      0.00         3
          7       1.00      0.25      0.40        16
          8       0.25      1.00      0.40        20
         10       0.00      0.00      0.00         3
         11       0.00      0.00      0.00         3
         12       0.00      0.00      0.00         3
         14       0.00      0.00      0.00         1
         15       0.00      0.00      0.00         1
         16       1.00      0.82      0.90        17
         17       0.00      0.00      0.00         4
         18       1.00      0.21      0.35        14
         19       0.00      0.00      0.00         3

avg / total       0.52      0.41      0.35       100



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [21]:
clf = SVC(kernel='linear').fit(X_train, y_train)

In [22]:
y_score = clf.predict(X_test)

In [23]:
n_right = 0
for i in range(len(y_score)):
    if y_score.tolist()[i] == y_test.tolist()[i]:
        n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(y_test)) * 100)))

Accuracy: 92.00%


In [24]:
print(classification_report(y_test, y_score))

             precision    recall  f1-score   support

          0       1.00      0.70      0.82        10
          3       0.00      0.00      0.00         1
          4       1.00      1.00      1.00         1
          6       1.00      1.00      1.00         3
          7       0.67      1.00      0.80        16
          8       1.00      1.00      1.00        20
         10       1.00      1.00      1.00         3
         11       1.00      1.00      1.00         3
         12       1.00      1.00      1.00         3
         14       1.00      1.00      1.00         1
         15       0.00      0.00      0.00         1
         16       1.00      1.00      1.00        17
         17       1.00      0.75      0.86         4
         18       1.00      1.00      1.00        14
         19       1.00      0.33      0.50         3

avg / total       0.93      0.92      0.91       100



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [25]:
df.head()

,author,file_name,text,author_LabelEncoded
0,Abraham Lincoln,Abraham Lincoln___Lincoln Letters.txt,LINCOLN LETTERS By Abraham Lincoln Published ...,0
1,Abraham Lincoln,Abraham Lincoln___Lincoln's First Inaugural Ad...,Lincoln First Inaugural Address March 4 1861 ...,0
2,Abraham Lincoln,Abraham Lincoln___Lincoln's Gettysburg Address...,Lincoln Gettysburg Address given November 19 ...,0
3,Abraham Lincoln,"Abraham Lincoln___Lincoln's Inaugurals, Addres...",Longman English Classics LINCOLN S INAUGURALS...,0
4,Abraham Lincoln,Abraham Lincoln___Lincoln's Second Inaugural A...,Lincoln Second Inaugural Address March 4 1865...,0


### Bag Of Word

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

In [27]:
count_vectorizer = CountVectorizer(stop_words='english')
XText_CountVectorizer = count_vectorizer.fit_transform(df['text'])

In [28]:
XText_CountVectorizer.shape[1]

133346

In [29]:
X_train, X_test, y_train, y_test = train_test_split(XText_CountVectorizer, df.author_LabelEncoded, random_state = 0)

In [30]:
clf = MultinomialNB().fit(X_train, y_train)

In [31]:
y_score = clf.predict(X_test)

In [32]:
n_right = 0
for i in range(len(y_score)):
    if y_score.tolist()[i] == y_test.tolist()[i]:
        n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(y_test)) * 100)))

Accuracy: 89.00%


In [33]:
print(classification_report(y_test, y_score))

             precision    recall  f1-score   support

          0       0.91      1.00      0.95        10
          3       0.00      0.00      0.00         1
          4       1.00      1.00      1.00         1
          6       1.00      0.33      0.50         3
          7       0.83      0.94      0.88        16
          8       1.00      0.80      0.89        20
         10       0.75      1.00      0.86         3
         11       1.00      1.00      1.00         3
         12       1.00      1.00      1.00         3
         14       1.00      1.00      1.00         1
         15       0.00      0.00      0.00         1
         16       0.85      1.00      0.92        17
         17       1.00      0.75      0.86         4
         18       0.82      1.00      0.90        14
         19       1.00      0.67      0.80         3

avg / total       0.89      0.89      0.88       100



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [34]:
clf = SVC(kernel='linear').fit(X_train, y_train)

In [35]:
y_score = clf.predict(X_test)

In [36]:
y_score.shape

(100,)

In [37]:
n_right = 0
for i in range(len(y_score)):
    if y_score.tolist()[i] == y_test.tolist()[i]:
        n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(y_test)) * 100)))

Accuracy: 77.00%


In [38]:
print(classification_report(y_test.tolist(), y_score.tolist()))

             precision    recall  f1-score   support

          0       0.82      0.90      0.86        10
          3       0.00      0.00      0.00         1
          4       1.00      1.00      1.00         1
          5       0.00      0.00      0.00         0
          6       0.40      0.67      0.50         3
          7       0.81      0.81      0.81        16
          8       0.94      0.85      0.89        20
         10       1.00      1.00      1.00         3
         11       0.60      1.00      0.75         3
         12       1.00      0.67      0.80         3
         13       0.00      0.00      0.00         0
         14       0.50      1.00      0.67         1
         15       0.00      0.00      0.00         1
         16       0.92      0.71      0.80        17
         17       1.00      0.50      0.67         4
         18       0.65      0.79      0.71        14
         19       0.33      0.33      0.33         3

avg / total       0.80      0.77      0.77  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### Word Embedding

In [39]:
### Create sequence
vocabulary_size = XText_CountVectorizer.shape[1]
num_of_author = len(df['author_LabelEncoded'].unique())

tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=5000)

In [40]:
X_train, X_test, y_train, y_test = train_test_split(data, np_utils.to_categorical(df['author_LabelEncoded']), random_state = 0)

In [ ]:
model = Sequential()
model.add(Embedding(vocabulary_size, 256, input_length = data.shape[1]))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_of_author, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=5)

Epoch 1/10
 60/300 [=====>........................] - ETA: 5:48 - loss: 2.9819 - acc: 0.1500

In [ ]:
y_score = model.predict(X_test)
y_score = [[1 if i == max(sc) else 0 for i in sc] for sc in y_score]
n_right = 0
for i in range(len(y_score)):
    if all(y_score[i][j] == np.array(y_test)[i][j] for j in range(len(y_score[i]))):
        n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(y_test)) * 100)))